In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.0


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import math, os, random, cv2
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
BASE_PATH = 'C:/Users/sonya/PythonProjects/image_classifier/input/wikiart-gangogh-creating-art-gan/'
abstract_paint        = os.path.join(BASE_PATH, 'abstract')
animal_paint          = os.path.join(BASE_PATH, 'animal-painting')
cityspace_paint       = os.path.join(BASE_PATH, 'cityscape')
figurative_paint      = os.path.join(BASE_PATH, 'figurative')
flower_paint_paint    = os.path.join(BASE_PATH, 'flower-painting')
genre_painting_paint  = os.path.join(BASE_PATH, 'genre-painting')
landscape_paint       = os.path.join(BASE_PATH, 'landscape')
marina_paint          = os.path.join(BASE_PATH, 'marina')
myth_paint_paint      = os.path.join(BASE_PATH, 'mythological-painting')
portrait_paint        = os.path.join(BASE_PATH, 'portrait')
religious_paint_paint = os.path.join(BASE_PATH, 'religious-painting')
still_life_paint      = os.path.join(BASE_PATH, 'still-life')
symbolic_paint        = os.path.join(BASE_PATH, 'symbolic-painting')

print(f'Abstract images      : {len(os.listdir(abstract_paint))}')
print(f'Animal images        : {len(os.listdir(animal_paint))}')
print(f'Cityspace images     : {len(os.listdir(cityspace_paint))}')
print(f'Figurative images    : {len(os.listdir(figurative_paint))}')
print(f'Flower images        : {len(os.listdir(flower_paint_paint))}')
print(f'Genre images         : {len(os.listdir(genre_painting_paint))}')
print(f'Landscape images     : {len(os.listdir(landscape_paint))}')
print(f'Marina images        : {len(os.listdir(marina_paint))}')
print(f'Mythological images  : {len(os.listdir(myth_paint_paint))}')
print(f'Portrait images      : {len(os.listdir(portrait_paint))}')
print(f'Religious images     : {len(os.listdir(religious_paint_paint))}')
print(f'Still Life images    : {len(os.listdir(still_life_paint))}')
print(f'Symbolic images      : {len(os.listdir(symbolic_paint))}')

Abstract images      : 15023
Animal images        : 1799
Cityspace images     : 6600
Figurative images    : 4500
Flower images        : 1800
Genre images         : 14998
Landscape images     : 15000
Marina images        : 1800
Mythological images  : 2100
Portrait images      : 15000
Religious images     : 8394
Still Life images    : 3000
Symbolic images      : 3000


In [4]:
permission = 224

def prepare_image(path):
    image = Image.open(path).convert('L')
    data = np.asarray(image)
    res_image = cv2.resize(data, dsize=(permission, permission), interpolation=cv2.INTER_CUBIC)
    normalized_image = res_image / 255.0 # нормализация от 0 до 1
    
    return normalized_image

In [5]:
train_images = np.empty([0,permission,permission])
test_images = np.empty([0,permission,permission])
validation_images = np.empty([0,permission,permission])
train_labels = np.empty([0], dtype = int)
test_labels = np.empty([0], dtype = int)
validation_labels = np.empty([0], dtype = int)


directory_train = 'C:/Users/sonya/PythonProjects/image_classifier/input/test'
count_class_names = 0
test_percent = 0.15
validation_percent = 0.15

for directory_name in os.listdir(directory_train):
    folder = os.path.join(directory_train, directory_name)
    len_folder = len(os.listdir(folder))
    image_count = 0
    validation_count = 0
    for image_name in os.listdir(folder):
        image_path = folder + '/' + image_name
        try:            
            if (image_count <= test_percent * len_folder):
                image_count += 1
                test_images = np.insert(test_images, test_images.shape[0], prepare_image(image_path), axis=0)
                test_labels = np.insert(test_labels, test_labels.shape[0], count_class_names)
            elif (validation_count <= validation_percent * len_folder):
                validation_count += 1
                validation_images = np.insert(validation_images, validation_images.shape[0], prepare_image(image_path), axis=0)
                validation_labels = np.insert(validation_labels, validation_labels.shape[0], count_class_names)
            else:
                train_images = np.insert(train_images, train_images.shape[0], prepare_image(image_path), axis=0)
                train_labels = np.insert(train_labels, train_labels.shape[0], count_class_names)
        except Exception as error:
            print(image_path, error)
            
    count_class_names += 1

In [6]:
labels_count = max(train_labels) + 1
print(labels_count)

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(permission, permission)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(labels_count, activation=tf.nn.softmax)
])

model.compile(optimizer = tf.optimizers.Adam(), 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

3


In [7]:
model.fit(train_images, 
          train_labels, 
          epochs = 5,
          validation_data = (validation_images, validation_labels))
          # steps_per_epoch = 100

Epoch 1/5
25/25 [==============================] - 1s 28ms/step - loss: 8.1348 - accuracy: 0.4008 - val_loss: 1.6504 - val_accuracy: 0.4826
Epoch 2/5
25/25 [==============================] - 1s 24ms/step - loss: 1.7737 - accuracy: 0.4475 - val_loss: 1.6252 - val_accuracy: 0.3779
Epoch 3/5
25/25 [==============================] - 1s 26ms/step - loss: 1.8197 - accuracy: 0.4425 - val_loss: 1.5131 - val_accuracy: 0.4012
Epoch 4/5
25/25 [==============================] - 1s 26ms/step - loss: 2.6870 - accuracy: 0.4210 - val_loss: 2.4729 - val_accuracy: 0.2558
Epoch 5/5
25/25 [==============================] - 1s 26ms/step - loss: 2.3368 - accuracy: 0.4614 - val_loss: 3.8874 - val_accuracy: 0.3605


In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

6/6 [==============================] - 0s 3ms/step - loss: 4.0137 - accuracy: 0.4128 
Test accuracy: 0.4127906858921051


In [9]:
predict_image_path = "C:/Users/sonya/PythonProjects/image_classifier/input/wikiart-gangogh-creating-art-gan/animal-painting/11084f51062008cc8184b4b8a48ccd14c.jpg"
array_image = np.empty([0, permission, permission])
array_image = np.insert(array_image, array_image.shape[0], prepare_image(predict_image_path), axis=0)
predictions = model.predict(array_image)
print(predictions)

[[3.3600031e-06 9.0354049e-01 9.6456215e-02]]


In [10]:
image_size = (permission, permission)
img_shape = image_size + (3, )
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')